!conda install numpy scipy scikit-learn pandas joblib pytorch

!pip install deap update_checker tqdm stopit xgboost

!pip install tpot

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import timeit 

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\lostx\Downloads\Assignments\Prepped_Data.csv")
df.head()

importing and making sure my data is intact as usual

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tenure              7032 non-null   int64  
 1   PhoneService        7032 non-null   int64  
 2   Contract            7032 non-null   int64  
 3   PaymentMethod       7032 non-null   int64  
 4   MonthlyCharges      7032 non-null   float64
 5   TotalCharges        7032 non-null   float64
 6   Churn               7032 non-null   int64  
 7   Total_Tenure_ratio  7032 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 439.6 KB


In [17]:
features = df.drop('Churn', axis=1)
targets = df['Churn']

x_train, x_test, y_train, y_test = train_test_split(features, targets, stratify=targets, random_state=42)

I chose churn as my target for the dataset, as it is the variable we are trying to find correlation to, and then find ways to prevent for our corperate partners.

In [19]:
%%time
tpot = TPOTClassifier(generations=5, population_size=50, cv=5,random_state=42, scoring='accuracy', verbosity=2, n_jobs=-1)

tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7961696808367131

Generation 2 - Current best internal CV score: 0.7961696808367131

Generation 3 - Current best internal CV score: 0.7961696808367131

Generation 4 - Current best internal CV score: 0.7974970547766577

Generation 5 - Current best internal CV score: 0.7974970547766577

Best pipeline: RandomForestClassifier(MinMaxScaler(input_matrix), bootstrap=True, criterion=entropy, max_features=0.2, min_samples_leaf=8, min_samples_split=19, n_estimators=100)
0.7923777019340159
CPU times: total: 41.5 s
Wall time: 5min 20s


This took less time than I thought it would, but i am happy to see the autoML comparing the data for me as opposed to what we have done in previous weeks.

In [23]:
predictions = tpot.predict(x_test)
predictions

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

Below is my comparison between the Prediction and my actual for the test data.

In [26]:
print('Predictions for test data set')
print(predictions)
print('Actuals for test data set')
print(y_test)

Predictions for test data set
[1 0 0 ... 0 0 1]
Actuals for test data set
942     1
503     0
4848    1
4143    0
5875    1
       ..
3678    0
6245    0
5496    0
3837    1
5719    1
Name: Churn, Length: 1758, dtype: int64


In [28]:
from sklearn.metrics import accuracy_score
print(f'Accuracy of the TPOT predictions: {accuracy_score(y_test,predictions)}')

Accuracy of the TPOT predictions: 0.7923777019340159


Next, i will save my TPOT model

Next, we want to save our trained model so we can use it in a Python file later. Afterall, having a model that is only available on your local machine isn't idea in a corporate setting. You will want to by able to deploy your model into a production environment. 

Most production environments don't utilize Jupyter notebooks. The `export()` command will create a native python file for us.

In [34]:
tpot.export('tpot_churn_pipeline.py')

In [36]:
from IPython.display import Code

Code('tpot_churn_pipeline.py')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: 0.7974970547766577
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.2, min_samples_leaf=8, min_samples_split=19, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

# Summary

I installed tpot to use as my AutoML for my churn dataset, and then ran multiple generations testing for the best accuracy in the set of tests, which was 0.7923777019340159. I'm not sure if this was any more accurate than when the action was performed by hand, but it took less time, which is great. I then exported my .py file and uploaded it to the GitHub repository.